In [33]:
from bs4 import BeautifulSoup as bbs
import requests as rq

import os
import sys
import json
import pandas as pd
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [34]:
def web_driver(baseurl, browser='firefox'):
    if 'linux' in sys.platform.lower():
        if browser.lower() == 'firefox':
            driver = webdriver.Firefox()
        else:
            driver = webdriver.Chrome()
    else:
        path = os.path.join(os.getcwd(), 'driver')
        if browser == 'FireFox':
            driver = webdriver.Firefox(path)
        else:
            driver = webdriver.Chrome(path)

    wait = WebDriverWait(driver, 10)
    driver.get(baseurl)
    return driver

def extract(driver):
    res_data = bbs(driver.page_source, 'html.parser')

    all_data = []
    param_name = ['Bed', 'Bath', 'Stories', 'Building Sqft',
                  'Garage(s) / Attached', 'Lot Sqft', 'Built', 'Private Pool']
    for i in res_data.findAll('div', {'class': 'prop_item'}):
        all_param = {}

        price = i.find('div', {'class': 'price'}).text.strip()
        address = i.find('a', {'class': 'address'}).text.strip()
        params = [j.text.replace('.', '').strip()
                  for j in i.findAll('div', {'class': 'mpf_item'})]
        all_param = {m: n for m, n in zip(param_name, params)}
        all_param['Price'] = price
        all_param['Address'] = address
        all_param['Comment'] = i.find('p').text
        all_data.append(all_param)
    return all_data

In [35]:
baseurl = 'https://www.har.com/'
driver = web_driver(baseurl)

# Select Filter Specification in Browser Window

In [36]:
all_main_data = []
all_main_data += extract(driver)

In [37]:
pagination = driver.find_elements('css selector', "div[class='pagination")[0]
links = pagination.find_elements('css selector', "a[href='javascript:void(0)']")[1:]

for i in range(len(links)):
    link = links[i]
    link.click()
    time.sleep(3)
    all_main_data += extract(driver)
    pagination = driver.find_elements('css selector', "div[class='pagination")[0]
    links = pagination.find_elements('css selector', "a[href='javascript:void(0)']")[1:]

In [102]:
driver.close()

In [38]:
df =pd.DataFrame(all_main_data)

In [39]:
df.head()

,Bed,Bath,Price,Address,Comment,Stories,Building Sqft,Garage(s) / Attached,Lot Sqft,Built,Private Pool
0,"2,200 Acre(s)","Road Frontage: ASPHAL,FMROA","$ 19,950,000","1352 FM 1397, Texarkana, TX 33471","Lot & Acreage - Farm/Ranch Property, in None ...",NaN,NaN,NaN,NaN,NaN,NaN
1,6 Bed(s),6 Full & 1 Half Bath(s),"$ 699,900","40 Knotty Pine Pl, Texarkana, TX 75503","Single Family Property, Traditional Style",180 Acre(s),Built in 1987,No Private Pool,NaN,NaN,NaN
2,NaN,NaN,"$ 650,000","4035 New Boston Rd, Texarkana, TX 75501","Residential Lots Property,",NaN,NaN,NaN,NaN,NaN,NaN
3,3 Bed(s),3 Full & 1 Half Bath(s),"$ 615,000","7 Shilling Pl, Texarkana, TX 75503","Single Family Property, Colonial Style","4,222 Building Sqft",463 Acre(s),Built in 1994,No Private Pool,NaN,NaN
4,5 Bed(s),3 Full & 1 Half Bath(s),"$ 593,500","8 Forrest Brook Ln, Texarkana, TX 75503","Residential - Single Family Property, Colonial...",2 Stories,"5,547 Building Sqft",3 Garage(s) / Attached,113 Acre(s),Built in 1993,No Private Pool
